In [1]:
import numpy as np

from sudoku_stuff import *

# Resolviendo Sudokus con Algoritmos Genéticos
Inteligencia Artificial - Facundo A. Lucianna - CEIA - FIUBA

En las notebooks anteriores, intentamos resolver sudokus utilizando los algoritmos de gradiente descendente, simulated annealing y búsqueda local beam. Vimos que los mejores resultados los encontramos con **búsqueda local beam**. Ahora veamos cómo nos va con los **algoritmos genéticos**.

Como vimos en los videos, el algoritmo genético ya no es un enfoque tan de búsqueda local, sino que introduce el concepto de reproducción y mutación. Ahora, los nuevos estados que se generen serán producto de la reproducción de sus padres.

Resolvamos este problema:

<div>
<img src="./sudoku_7.png" width="300"/>
</div>

El diccionario de **celdas fijas** quedaría de la siguiente forma:

In [2]:
fixed_squares = {
    'A1': 3, 'A3': 4, 'A4': 5, 'A5': 6, 'A7': 9,
    'B1': 1, 'B2': 8, 'B3': 5, 'B6': 9, 'B7': 7,
    'C5': 7, 'C6': 8, 'C7': 4, 'C8': 1, 'C9': 5,
    'D2': 2, 'D5': 1, 'D8': 4, 'D9': 9,
    'E2': 4, 'E3': 9, 'E5': 5, 
    'F3': 1, "F4": 9, "F5": 8, "F7": 6, "F8": 7,
    'G1': 4, 'G2': 9, 'G5': 3, 'G9': 7, 
    'H2': 1, 'H3': 8, 'H4': 7, 'H5': 4, 'H6': 5, 'H9': 6,
    'I8': 8,
}

Y la solución que tenemos, que vamos a usar para verificar al final de todo es:

In [3]:
solution = {
    'A1': 3, 'A2': 7, 'A3': 4, 'A4': 5, 'A5': 6, 'A6': 1, 'A7': 9, 'A8': 2, 'A9': 8,
    'B1': 1, 'B2': 8, 'B3': 5, 'B4': 4, 'B5': 2, 'B6': 9, 'B7': 7, 'B8': 6, 'B9': 3,
    'C1': 9, 'C2': 6, 'C3': 2, 'C4': 3, 'C5': 7, 'C6': 8, 'C7': 4, 'C8': 1, 'C9': 5,
    'D1': 8, 'D2': 2, 'D3': 7, 'D4': 6, 'D5': 1, 'D6': 3, 'D7': 5, 'D8': 4, 'D9': 9,
    'E1': 6, 'E2': 4, 'E3': 9, 'E4': 2, 'E5': 5, 'E6': 7, 'E7': 8, 'E8': 3, 'E9': 1,
    'F1': 5, 'F2': 3, 'F3': 1, 'F4': 9, 'F5': 8, 'F6': 4, 'F7': 6, 'F8': 7, 'F9': 2,
    'G1': 4, 'G2': 9, 'G3': 6, 'G4': 8, 'G5': 3, 'G6': 2, 'G7': 1, 'G8': 5, 'G9': 7,
    'H1': 2, 'H2': 1, 'H3': 8, 'H4': 7, 'H5': 4, 'H6': 5, 'H7': 3, 'H8': 9, 'H9': 6,
    'I1': 7, 'I2': 5, 'I3': 3, 'I4': 1, 'I5': 9, 'I6': 6, 'I7': 2, 'I8': 8, 'I9': 4,
}

In [4]:
print_state(solution)

*---------+---------+---------*
| 3  7  4 | 5  6  1 | 9  2  8 |
| 1  8  5 | 4  2  9 | 7  6  3 |
| 9  6  2 | 3  7  8 | 4  1  5 |
*---------+---------+---------*
| 8  2  7 | 6  1  3 | 5  4  9 |
| 6  4  9 | 2  5  7 | 8  3  1 |
| 5  3  1 | 9  8  4 | 6  7  2 |
*---------+---------+---------*
| 4  9  6 | 8  3  2 | 1  5  7 |
| 2  1  8 | 7  4  5 | 3  9  6 |
| 7  5  3 | 1  9  6 | 2  8  4 |
*---------+---------+---------*


## Implementando herramientas para aplicar Algoritmos Genéticos en la resolución de Sudokus

Un algoritmo genético es una variante de la búsqueda local beam estocástica en la que los estados sucesores se generan combinando dos estados padres (reproducción). Para implementar este algoritmo, debemos definir varios elementos:

- Forma de codificar el estado como un string para definir el cromosoma.
- Implementar la reproducción
- Implementar la mutación
- La función de idoneidad, que utilizaremos como la función de costo que hemos venido usando.

### Cromosoma

Primero, veamos cómo podemos codificar un estado particular del Sudoku en un cromosoma, que es un string que luego podemos manipular para realizar la reproducción.

En este caso, dado que un Sudoku está formado por números, representaremos todos los números que llenan el Sudoku como una sola cadena de caracteres, aplanando a nivel de filas.

Veamos un ejemplo de la implementación para que quede más claro, utilizando la solución que definimos previamente.

In [14]:
# Las funciones que implementan los distintos elementos para resolver con algortimos geneticos estan definido en genetic.py
from genetic import *

In [15]:
print_state(solution)

*---------+---------+---------*
| 3  7  4 | 5  6  1 | 9  2  8 |
| 1  8  5 | 4  2  9 | 7  6  3 |
| 9  6  2 | 3  7  8 | 4  1  5 |
*---------+---------+---------*
| 8  2  7 | 6  1  3 | 5  4  9 |
| 6  4  9 | 2  5  7 | 8  3  1 |
| 5  3  1 | 9  8  4 | 6  7  2 |
*---------+---------+---------*
| 4  9  6 | 8  3  2 | 1  5  7 |
| 2  1  8 | 7  4  5 | 3  9  6 |
| 7  5  3 | 1  9  6 | 2  8  4 |
*---------+---------+---------*


In [16]:
# obtain_chromosome_sudoku devuelve el cromosoma del sudoku dado un estado (formado por un diccionario con las celdas
chromosome_solution = obtain_chromosome_sudoku(solution, obtain_all_cells())

In [17]:
print(chromosome_solution)

374561928185429763962378415827613549649257831531984672496832157218745396753196284


Con esta función, podemos asegurarnos de que, por ejemplo, cuando realicemos una mutación, esas posiciones fijas no sean modificadas.

Por último, en lo que respecta al cromosoma, necesitamos una función que, al pasarle un cromosoma, nos devuelva un estado. Es decir, el proceso de "nacimiento" de nuevos estados. Para esto, podemos usar la función `obtain_fixed_pos_in_chromosome_sudoku()`. Veamos un ejemplo:

Tomemos el cromosoma que calculamos anteriormente y modifiquemos el número en el índice `1`:.

In [19]:
pos_fixed = obtain_fixed_pos_in_chromosome_sudoku(fixed_squares, obtain_all_cells())

print(pos_fixed)

[0, 2, 3, 4, 6, 9, 10, 11, 14, 15, 22, 23, 24, 25, 26, 28, 31, 34, 35, 37, 38, 40, 47, 48, 49, 51, 52, 54, 55, 58, 62, 64, 65, 66, 67, 68, 71, 79]


Entonces con esta función podemos usarlo para saber cuando por ejemplo realicemos una mutuación, esas posiciones no pueden ser modificadas.

Por ultimo, en lo que respecta al cromosoma, necesitamos una funcion que le pasamos un cromosoma, y nos devuelva un estado, es decir el proceso de nacimiento de nuevos estados.
Esto lo podemos realizar usando `obtain_sibling_from_chromosome_sudoku()`. Veamos un ejemplo:

Tomemos el cromosoma que calculamos anteriormente y modifiquemos el numero del indice `1:`

In [22]:
new_chromosome = chromosome_solution[:1] + '1' + chromosome_solution[2:]
print(new_chromosome)

314561928185429763962378415827613549649257831531984672496832157218745396753196284


Obtengamos ahora el estado de este cromosoma:

In [23]:
new_state = obtain_sibling_from_chromosome_sudoku(new_chromosome, obtain_all_cells())
print_state(new_state)

*---------+---------+---------*
| 3  1  4 | 5  6  1 | 9  2  8 |
| 1  8  5 | 4  2  9 | 7  6  3 |
| 9  6  2 | 3  7  8 | 4  1  5 |
*---------+---------+---------*
| 8  2  7 | 6  1  3 | 5  4  9 |
| 6  4  9 | 2  5  7 | 8  3  1 |
| 5  3  1 | 9  8  4 | 6  7  2 |
*---------+---------+---------*
| 4  9  6 | 8  3  2 | 1  5  7 |
| 2  1  8 | 7  4  5 | 3  9  6 |
| 7  5  3 | 1  9  6 | 2  8  4 |
*---------+---------+---------*


### Reproducción

La reproducción la decidimos realizar mediante la siguiente operación. Dado dos cromosomas, seleccionamos un punto de corte al azar en el string. Usando ese punto de corte, concatenamos una parte del cromosoma de un padre con la del otro. Esta reproducción generará dos hijos, siendo cada uno el complemento del otro.

Esta reproduccion esta implementada en la función `reproduction_sudoku()`. Veamos un ejemplo de uso: 

In [26]:
# Generamos dos estados al azar 
state_1 = init_state(fixed_squares)
state_2 = init_state(fixed_squares)

print("Estado 1")
print_state(state_1)
print("Estado 2")
print_state(state_2)

Estado 1
*---------+---------+---------*
| 3  7  4 | 5  6  6 | 9  3  3 |
| 1  8  5 | 7  3  9 | 7  5  7 |
| 4  1  4 | 9  7  8 | 4  1  5 |
*---------+---------+---------*
| 3  2  7 | 7  1  5 | 4  4  9 |
| 4  4  9 | 3  5  7 | 8  9  6 |
| 2  9  1 | 9  8  7 | 6  7  7 |
*---------+---------+---------*
| 4  9  9 | 7  3  4 | 5  9  7 |
| 2  1  8 | 7  4  5 | 7  3  6 |
| 1  7  5 | 3  2  8 | 6  8  4 |
*---------+---------+---------*
Estado 2
*---------+---------+---------*
| 3  8  4 | 5  6  5 | 9  9  1 |
| 1  8  5 | 2  2  9 | 7  9  9 |
| 7  6  6 | 2  7  8 | 4  1  5 |
*---------+---------+---------*
| 5  2  7 | 5  1  8 | 5  4  9 |
| 1  4  9 | 6  5  4 | 4  8  6 |
| 4  5  1 | 9  8  7 | 6  7  1 |
*---------+---------+---------*
| 4  9  7 | 7  3  4 | 9  1  7 |
| 8  1  8 | 7  4  5 | 1  6  6 |
| 3  7  8 | 7  2  9 | 6  8  5 |
*---------+---------+---------*


In [27]:
# Obtenemos sus cromosomas
chromosome_state_1 = obtain_chromosome_sudoku(state_1, obtain_all_cells())
chromosome_state_2 = obtain_chromosome_sudoku(state_2, obtain_all_cells())

print("Estado 1")
print(chromosome_state_1)
print("Estado 2")
print(chromosome_state_2)

Estado 1
374566933185739757414978415327715449449357896291987677499734597218745736175328684
Estado 2
384565991185229799766278415527518549149654486451987671497734917818745166378729685


In [29]:
# Realizamos la reproducción
chromosome_sibling_1, chromosome_sibling_2 = reproduction_sudoku(chromosome_state_1, chromosome_state_2)

print("Estado hijo 1")
print(chromosome_sibling_1)
print("Estado hijo 2")
print(chromosome_sibling_2)

Estado hijo 1
374566933185739757414978415327715449449357896291987677499734517818745166378729685
Estado hijo 2
384565991185229799766278415527518549149654486451987671497734997218745736175328684


In [30]:
# Obtengamos a los hijos
sibling_1 = obtain_sibling_from_chromosome_sudoku(chromosome_sibling_1, obtain_all_cells())
sibling_2 = obtain_sibling_from_chromosome_sudoku(chromosome_sibling_2, obtain_all_cells())

print("Estado hijo 1")
print_state(sibling_1)
print("Estado hijo 2")
print_state(sibling_2)

Estado hijo 1
*---------+---------+---------*
| 3  7  4 | 5  6  6 | 9  3  3 |
| 1  8  5 | 7  3  9 | 7  5  7 |
| 4  1  4 | 9  7  8 | 4  1  5 |
*---------+---------+---------*
| 3  2  7 | 7  1  5 | 4  4  9 |
| 4  4  9 | 3  5  7 | 8  9  6 |
| 2  9  1 | 9  8  7 | 6  7  7 |
*---------+---------+---------*
| 4  9  9 | 7  3  4 | 5  1  7 |
| 8  1  8 | 7  4  5 | 1  6  6 |
| 3  7  8 | 7  2  9 | 6  8  5 |
*---------+---------+---------*
Estado hijo 2
*---------+---------+---------*
| 3  8  4 | 5  6  5 | 9  9  1 |
| 1  8  5 | 2  2  9 | 7  9  9 |
| 7  6  6 | 2  7  8 | 4  1  5 |
*---------+---------+---------*
| 5  2  7 | 5  1  8 | 5  4  9 |
| 1  4  9 | 6  5  4 | 4  8  6 |
| 4  5  1 | 9  8  7 | 6  7  1 |
*---------+---------+---------*
| 4  9  7 | 7  3  4 | 9  9  7 |
| 2  1  8 | 7  4  5 | 7  3  6 |
| 1  7  5 | 3  2  8 | 6  8  4 |
*---------+---------+---------*


### Mutuación

Un aspecto importante en los algoritmos genéticos es la aleatoriedad introducida por la mutación. La idea es que, por azar, se modifique una parte del cromosoma, de tal forma que, por simple casualidad, podamos encontrar un estado que se adapte mejor. En este caso, cuando mutamos, cambiamos una posición del cromosoma que no sea fija por un valor al azar.

Para definir si se realiza o no la mutación, decidimos usar una "temperatura", la cual, cuanto más alta sea, mayor será la probabilidad de que el cromosoma mute. Por el contrario, cuando la temperatura sea baja, la probabilidad de mutación será menor. Implementamos algo similar a simulated annealing, en donde aceptamos la mutación si, al generar un valor aleatorio entre 0 y 1, este es menor que `exp(-1 * (1 / temperatura))`.

Esto lo implementamos en la función `mutate_chromosome_sudoku_with_temperature()`. Veamos un ejemplo de esta función con uno de los hijos obtenidos y utilizando una temperatura muy alta para forzar la mutación.

In [31]:
print("Estado hijo 1")
print_state(sibling_1)

Estado hijo 1
*---------+---------+---------*
| 3  7  4 | 5  6  6 | 9  3  3 |
| 1  8  5 | 7  3  9 | 7  5  7 |
| 4  1  4 | 9  7  8 | 4  1  5 |
*---------+---------+---------*
| 3  2  7 | 7  1  5 | 4  4  9 |
| 4  4  9 | 3  5  7 | 8  9  6 |
| 2  9  1 | 9  8  7 | 6  7  7 |
*---------+---------+---------*
| 4  9  9 | 7  3  4 | 5  1  7 |
| 8  1  8 | 7  4  5 | 1  6  6 |
| 3  7  8 | 7  2  9 | 6  8  5 |
*---------+---------+---------*


In [33]:
# Realizamos la mutación
chromosome_mutated = mutate_chromosome_sudoku_with_temperature(chromosome_sibling_1, pos_fixed, temperature=100000)

In [34]:
# Obtengamos al hijo mutado
sibling_mutated = obtain_sibling_from_chromosome_sudoku(chromosome_mutated, obtain_all_cells())

print("Estado mutado")
print_state(sibling_mutated)

Estado mutado
*---------+---------+---------*
| 3  7  4 | 5  6  6 | 9  3  3 |
| 1  8  5 | 7  3  9 | 7  5  7 |
| 4  1  4 | 9  7  8 | 4  1  5 |
*---------+---------+---------*
| 3  2  7 | 7  1  5 | 4  4  9 |
| 4  4  9 | 3  5  7 | 1  9  6 |
| 2  9  1 | 9  8  7 | 6  7  7 |
*---------+---------+---------*
| 4  9  9 | 7  3  4 | 5  1  7 |
| 8  1  8 | 7  4  5 | 1  6  6 |
| 3  7  8 | 7  2  9 | 6  8  5 |
*---------+---------+---------*


## Implementación de Búsqueda con Algoritmos Genéticos

Ya con todo lo necesario definido e implementado para el caso particular de Sudokus, armemos nuestra implementación de búsqueda con algoritmos genéticos. La última cosa que nos quedaba por definir es la función de idoneidad para determinar a los mejores, pero para esto ya contamos con nuestra función de costo.

**Nota**: Todas las decisiones tomadas para el diseño, como la reproducción, la conformación del cromosoma, etc., son decisiones de diseño que son puramente arbitrarias y dependen de la creatividad del diseñador. Otras decisiones de diseño pueden dar resultados muy diferentes.

Veamos la implementación, leyendo cada comentario con atención:

In [5]:
import itertools

In [6]:
def genetic_algorithm_sudoku(initial_generation: list, fixed_squares: dict, max_iterations: int = 50, initial_temperature: float = 100) -> tuple:
    """
    Realiza la optimización del Sudoku utilizando un algoritmo genético.

    Args:
        initial_generation (list): Lista con estados iniciales del Sudoku.
        fixed_squares (dict): Diccionario que contiene las casillas fijas del Sudoku.
        max_iterations (int, optional): El número máximo de iteraciones permitidas. Por defecto es 100.
        initial_temperature (float, optional): La temperatura inicial para controlar la probabilidad de mutación. Por defecto es 0.1.

    Returns:
        dict: El mejor estado encontrado después de la optimización.
        float: El costo del mejor estado encontrado
        int: Numero indicando en que generación se encontró el mejor resultado
    """
    temperature = initial_temperature
    best_state = initial_generation[0]
    best_cost = cost_function(best_state)
    best_iteration = 0
    no_changes = 0

    # Determinamos el numero de la poblacion que vivirá cada generación
    number_population = len(initial_generation)
    current_population = initial_generation.copy()

    # Obtenemos las posiciones del sudoku que no pueden mutar
    squares = obtain_all_cells()
    not_valid_positions = obtain_fixed_pos_in_chromosome_sudoku(fixed_squares, squares)

    # Iteramos hasta max_iterations
    for iteration in range(max_iterations):

        no_changes += 1

        # Para cada uno de la generación, calculamos su función de costo
        actual_cost_list = [cost_function(state) for state in current_population]

        generate_print = False
        for index, cost in enumerate(actual_cost_list):
            # Nos quedamos como mejor estado al que mejore el valor de costo historico
            if cost < best_cost:
                no_changes = 0
                generate_print = True
                best_state = current_population[index]
                best_cost = cost
                best_iteration = iteration

        # Si encontramos un estado con costo 0, es que encontramos la solución.
        # Tambien, si pasamos muchas generaciones que no mejoran el mejor costo, significa que es dificil que 
        # podamos mejorar, y terminamos la ejecución.
        if best_cost == 0 or no_changes > 9:
            return best_state, best_cost, best_iteration

        if generate_print:
            print(f"El mejor costo es: {best_cost} en la iteración {iteration}")

        # Ordenamos a los estados en función del costo de menor a mayor (función de idoneidad)
        index_list = sorted(range(len(actual_cost_list)), key=lambda x: actual_cost_list[x])
        current_population = [current_population[k] for k in index_list]

        # Nos quedamos con solo un valor de estados igual a la población inicial que pasan a reproducirse
        # Acá extinguimos a los mas "debiles"
        current_population = current_population[:number_population]

        # Obtenemos los cromosomas de los estados
        all_chromosome = [obtain_chromosome_sudoku(state, squares) for state in current_population]

        # Generamos los hijos. Reproducimos a todos con todos
        sibling_chromosomes_list = []
        for chromosome_1, chromosome_2 in itertools.combinations(all_chromosome, 2):
            offspring_1, offspring_2 = reproduction_sudoku(chromosome_1, chromosome_2)
            sibling_chromosomes_list.extend([offspring_1, offspring_2])

        # Vemos si algún hijo muta
        sibling_chromosomes_list = [mutate_chromosome_sudoku_with_temperature(chromosome, not_valid_positions, temperature)
                                    for chromosome in sibling_chromosomes_list]

        # De los padres usados para reproducir, los diezmamos.
        current_population = current_population[:number_population // 10]

        # Creamos la nueva generación de estados usando el cromosoma obtenido
        # Y los agregamos a la generación anterior diezmada. Es decir, mantenemos los mejores padres.
        current_population += [obtain_sibling_from_chromosome_sudoku(chromosome, squares) for chromosome in
                               sibling_chromosomes_list]

    # Si terminamos las iteraciones, retornamos el mejor resultado encontrado
    return best_state, best_cost, best_iteration

Ahora, veamos si podemos encontrar la solución en una única ejecución. Para ello, vamos a llamar a una función llamada `execute_search_evolution()`, a la cual le pasamos la implementación del algoritmo. Esta función inicializa varios estados de sudoku al azar, aplica la búsqueda y verifica si se ha alcanzado una solución o no.

In [7]:
from processing import execute_search_evolution

In [8]:
solution_bool, best_state, best_generation, _ = execute_search_evolution(0, genetic_algorithm_sudoku, fixed_squares, number_generation_initial=200)

El mejor costo es: 5.800000000000001 en la iteración 0
El mejor costo es: 5.050000000000001 en la iteración 1
El mejor costo es: 4.300000000000002 en la iteración 2
El mejor costo es: 3.5000000000000013 en la iteración 3
El mejor costo es: 3.100000000000001 en la iteración 4
El mejor costo es: 2.600000000000001 en la iteración 5
El mejor costo es: 2.200000000000001 en la iteración 6
El mejor costo es: 1.7000000000000006 en la iteración 7
El mejor costo es: 1.3 en la iteración 8
El mejor costo es: 0.8999999999999999 en la iteración 9
El mejor costo es: 0.6 en la iteración 10
El mejor costo es: 0.4 en la iteración 11
El mejor costo es: 0.2 en la iteración 13


In [9]:
print(f"Valor del mejor costo encontrado: {cost_function(best_state)}, el mejor estado se encontró en la {best_generation} generación")

print("Mejor estado encontrado:")
print_state(best_state)

Valor del mejor costo encontrado: 0.2, el mejor estado se encontró en la 13 generación
Mejor estado encontrado:
*---------+---------+---------*
| 3  7  4 | 5  6  1 | 9  2  8 |
| 1  8  5 | 4  2  9 | 7  6  3 |
| 9  6  2 | 3  7  8 | 4  1  5 |
*---------+---------+---------*
| 8  2  3 | 6  1  7 | 5  4  9 |
| 7  4  9 | 2  5  3 | 8  3  1 |
| 6  5  1 | 9  8  4 | 6  7  2 |
*---------+---------+---------*
| 4  9  6 | 8  3  2 | 1  5  7 |
| 2  1  8 | 7  4  5 | 3  9  6 |
| 5  3  7 | 1  9  6 | 2  8  4 |
*---------+---------+---------*


In [10]:
print("El estado encontrado es solución?")
if solution_bool:
    print("El estado que encontramos verifica que realmente es la solución")
else:
    print("El estado que encontramos no es solución")

El estado encontrado es solución?
El estado que encontramos no es solución


Vemos que no se encontró la solución. El algoritmo estuvo muy cerca, alcanzando un costo bajo y utilizando pocas generaciones, pero llegó a un punto en el que solo un par de números estaban mal ubicados (con solo 4 penalizaciones en las filas 5 y 6). El problema en esta implementación es que, rápidamente, los estados de bajo costo dominan las generaciones y, para salir de este estado de bajo costo, solo sería posible mediante una mutación que modifique justo algunos de esos valores. Sin embargo, esto es improbable, ya que solo se modifica un solo valor y, aunque este valor se ubique en el lugar correcto, puede seguir teniendo la misma penalización.

Se podrían implementar dos mejoras en esta implementación, que se dejan como ejercicio:

- Implementar un segundo tipo de mutación en el que, en lugar de cambiar un valor al azar, se intercambian números de posición.
- Entre un cierto número de generaciones, introducir nuevos estados totalmente formados al azar, o permitir que sobrevivan estados con alto costo, de tal manera que se introduzca variedad.

Como prueba final, vamos a ejecutar la búsqueda 100 veces y verificar cuántas veces llega a una solución. Para acelerar el proceso, aprovecharemos que tenemos **CPUs multinúcleo**.

Para ello, vamos a llamar a la función `parallel_sudoku_search()`, a la cual le pasaremos la función de búsqueda y el número de iteraciones que queremos realizar, indicandole que estamos usando un algoritmo de búsqueda que usa multiple estados.

In [11]:
from processing import parallel_sudoku_search

# Debemos llamar a la función de busqueda desde un archivo .py sino los threads no pueden recibir a la función desde la notebook directamente.
from search_methods import genetic_algorithm_sudoku

In [12]:
results = parallel_sudoku_search(genetic_algorithm_sudoku, fixed_squares, max_iterations=100, generation_method=True, number_generation_initial=200)

  0%|          | 0/100 [00:00<?, ?it/s]

Veamos si algún proceso encontró la solución:

In [13]:
show_solution = True
for res in results:
    # Acá nos devuelve el booleano de si encontró la solución o no
    is_solution = res[0]
    # Este es el últimos estado encontrado en esta iteración
    last_state = res[1]
    # Este es el estado desde donde partio
    initial_state = res[2]
    # Este es el identificador de cual iteración se obtuvo la solución
    process_id = res[-1]

    if is_solution:
        if show_solution:
            print_state(last_state)
            show_solution = False
        print(f"En la iteración {process_id} se encontró la solución.")

*---------+---------+---------*
| 3  7  4 | 5  6  1 | 9  2  8 |
| 1  8  5 | 4  2  9 | 7  6  3 |
| 9  6  2 | 3  7  8 | 4  1  5 |
*---------+---------+---------*
| 8  2  7 | 6  1  3 | 5  4  9 |
| 6  4  9 | 2  5  7 | 8  3  1 |
| 5  3  1 | 9  8  4 | 6  7  2 |
*---------+---------+---------*
| 4  9  6 | 8  3  2 | 1  5  7 |
| 2  1  8 | 7  4  5 | 3  9  6 |
| 7  5  3 | 1  9  6 | 2  8  4 |
*---------+---------+---------*
En la iteración 98 se encontró la solución.


Vemos que de las 100 ejecuciones, solo una vez se encontró la solución, mientras que en el caso de **Local Beam** los resultados fueron mucho más fructíferos. La forma en que obtenemos los vecinos resulta más apropiada que los métodos de reproducción y mutación implementados aquí.

Viendo el lado positivo, este comportamiento revela en qué aspectos es fuerte este algoritmo. Rápidamente encuentra mínimos, incluso más bajos que los otros algoritmos cuando estos fallan. Sin embargo, no es tan bueno para encontrar el mejor caso, como en el Sudoku, donde solo tenemos una única solución. Si el problema es más general, por ejemplo, cuando buscamos un estado que cumpla con algún criterio de optimización (como minimizar parámetros, error, energía o costo por debajo de un umbral), este algoritmo tiene mayor utilidad.

Siempre con la desventaja del gran trabajo de diseño previo que se requiere.